In [1]:
#!pip install -U pyTigerGraph

In [2]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd
import os

In [3]:
pfiles  = r'FETFiles'
os.listdir(pfiles)
fetg = pd.read_excel(pfiles + '/Graph.xlsx')

In [4]:
fetg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Index                38 non-null     int64 
 1   x_min                38 non-null     int64 
 2   y_min                38 non-null     int64 
 3   x_max                38 non-null     int64 
 4   Y_max                38 non-null     int64 
 5   text                 38 non-null     object
 6   side_node_idx        38 non-null     int64 
 7   side_node_text       10 non-null     object
 8   side_node_distance   38 non-null     int64 
 9   below_node_idx       38 non-null     int64 
 10  below_node_text      34 non-null     object
 11  below_node_distance  38 non-null     int64 
dtypes: int64(9), object(3)
memory usage: 3.7+ KB


In [5]:
textnode = fetg[['Index','text']]
sidenode = fetg[['Index','side_node_idx','Index','side_node_text']]
belownode = fetg[['Index','below_node_idx','Index','below_node_text']]

In [6]:
tfilename  = 'FETFiles/g_configs/textnode.csv'
snfilename = 'FETFiles/g_configs/snnode.csv'
dnfilename = 'FETFiles/g_configs/dnnode.csv'

In [7]:
# textnode['Index'].fillna(-1,inplace=True)
# sidenode['Index'].fillna(-1,inplace=True)
# belownode['Index'].fillna(-1,inplace=True)

# sidenode['side_node_idx'].fillna(-1,inplace=True)
# belownode['below_node_idx'].fillna(-1,inplace=True)

In [8]:
textnode.to_csv(tfilename,index=False)
sidenode.to_csv(snfilename,index=False)
belownode.to_csv(dnfilename,index=False)

In [9]:
# Connection parameters
hostName = "https://8b63e18824d24d80b8579495e8bdaf13.i.tgcloud.io"
userName = "user_1"
password = "Preetha_021"

In [10]:
graph_name = 'FET_New1'

In [11]:
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password,graphname=graph_name)

In [12]:
results = conn.gsql(
'''
CREATE GRAPH %s ()
''' % (graph_name)
)
    
print(results)

/Users/s/anaconda3/lib/python3.10/site-packages/pyTigerDriver/pyDriver.py:205: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl.PROTOCOL_SSLv23)


The graph FET_New1 is created.


In [13]:
conn.gsql("LS")

'---- Global vertices, edges, and all graphs\nVertex Types:\nEdge Types:\n\nGraphs:\n- Graph Person(Application:v, Inventor:v, has_inventor:e)\n- Graph FET_Graph(text:v, belownode:v, sidenode:v, sideedge:e, reverse_sideedge:e, sidetotext:e, reverse_sidetotext:e, belowedge:e, reverse_belowedge:e, belowtotext:e)\n- Graph imdb(Movie:v, Actor:v, Director:v, actor_movie:e, director_movie:e, movie_actor:e, movie_director:e)\n- Graph FET_New1()\nJobs:\n\n\nJSON API version: v2\nSyntax version: v2\n'

In [14]:
secret = conn.createSecret()

In [15]:
authToken = conn.getToken(secret)[0]

In [16]:
conn.getVertexCount("*")

{}

In [17]:
results = conn.gsql(
    ''' 
    USE GRAPH FET_New1
    
    # Create vertex and edge type 
    CREATE SCHEMA_CHANGE JOB FET_New2 FOR GRAPH FET_New1 {
        ADD VERTEX Text (PRIMARY_ID id INT, value STRING) WITH primary_id_as_attribute="true";
        ADD UNDIRECTED EDGE side_node_edge (FROM Text, TO Text, id INT, value STRING);
        ADD UNDIRECTED EDGE down_node_egde (FROM Text, TO Text, id INT, value STRING);
    }
    
    RUN SCHEMA_CHANGE JOB FET_New2
    '''
)

print (results)

Using graph 'FET_New1'
Successfully created schema change jobs: [FET_New2].
Kick off schema change job FET_New2
Doing schema change on graph 'FET_New1' (current version: 0)
Trying to add local vertex 'Text' to the graph 'FET_New1'.
Trying to add local edge 'side_node_edge' to the graph 'FET_New1'.
Trying to add local edge 'down_node_egde' to the graph 'FET_New1'.

Graph FET_New1 updated to new version 1
The job FET_New2 completes in 2.096 seconds!
Local schema change succeeded.


In [18]:
results = conn.gsql(
    '''
    USE GRAPH FET_New1

    CREATE LOADING JOB load_FET_New2 FOR GRAPH FET_New1 {
        DEFINE FILENAME text_file;
        DEFINE FILENAME side_node_file;
        DEFINE FILENAME below_node_file;

        
        
        LOAD text_file
            TO VERTEX Text VALUES ($0, $1)
            USING Header="true",  separator=",";
            
        LOAD side_node_file
            TO EDGE side_node_edge VALUES ($0,$1,$2,$3)
            USING Header="true", separator=",";

        LOAD below_node_file
            TO EDGE down_node_egde VALUES ($0,$1,$2,$3)
            USING Header="true", separator=",";       
    }
    '''
)

print (results)

Using graph 'FET_New1'
Successfully created loading jobs: [load_FET_New2].


In [19]:
uploadTextFile = conn.runLoadingJobWithFile(tfilename, "text_file", "load_FET_New2", ",")
print (uploadTextFile)

[{'sourceFileName': 'Online_POST', 'statistics': {'sourceFileName': 'Online_POST', 'parsingStatistics': {'fileLevel': {'validLine': 39}, 'objectLevel': {'vertex': [{'typeName': 'Text', 'validObject': 38, 'invalidPrimaryId': 1}], 'edge': [], 'deleteVertex': [], 'deleteEdge': []}}}}]


In [20]:
uploadSideNodeFile = conn.runLoadingJobWithFile(snfilename, "side_node_file", "load_FET_New2", ",")
print (uploadSideNodeFile)

[{'sourceFileName': 'Online_POST', 'statistics': {'sourceFileName': 'Online_POST', 'parsingStatistics': {'fileLevel': {'validLine': 39}, 'objectLevel': {'vertex': [], 'edge': [{'typeName': 'side_node_edge', 'validObject': 38, 'invalidPrimaryId': 1}], 'deleteVertex': [], 'deleteEdge': []}}}}]


In [21]:
uploadDownNodeFile = conn.runLoadingJobWithFile(dnfilename, "below_node_file", "load_FET_New2", ",")
print (uploadDownNodeFile)

[{'sourceFileName': 'Online_POST', 'statistics': {'sourceFileName': 'Online_POST', 'parsingStatistics': {'fileLevel': {'validLine': 39}, 'objectLevel': {'vertex': [], 'edge': [{'typeName': 'down_node_egde', 'validObject': 38, 'invalidPrimaryId': 1}], 'deleteVertex': [], 'deleteEdge': []}}}}]


In [22]:
vertices = conn.getVertexTypes()
total_count = 0
for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (Text : 28) 
Total node count:  28


In [23]:
conn.getVertexCount("*")

{'Text': 36}

In [24]:
conn.getEdgeCount("*")

{'side_node_edge': 38, 'down_node_egde': 38}

In [25]:
import pandas as pd

tom = conn.getVerticesById("Text", "3")

pd.json_normalize(tom)

,v_id,v_type,attributes.id,attributes.value
0,3,Text,3,Attn: Mr. Sundar


In [26]:
sideedge = conn.getEdges("Text", "29", "side_node_edge")

pd.json_normalize(sideedge)

,e_type,directed,from_id,from_type,to_id,to_type,attributes.id,attributes.value
0,side_node_edge,False,29,Text,30,Text,29,7500
1,side_node_edge,False,29,Text,28,Text,28,EUR


In [27]:
downedge = conn.getEdges("Text", "25", "down_node_egde")

pd.json_normalize(downedge)

,e_type,directed,from_id,from_type,to_id,to_type,attributes.id,attributes.value
0,down_node_egde,False,25,Text,25,Text,25,Annual Membership Fee 2022 (Circular 5335)
